In [1]:
# Data visualization
import matplotlib.pyplot as plt

# Data manipulation
import pandas as pd
import numpy as np

import glob, re, os, sys, random
from random import shuffle
import random
from time import time

# Vector representations and embeddings
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import gensim

# Modeling - Logistic, XGBOOST, SVM
from sklearn.model_selection import GridSearchCV, train_test_split, StratifiedShuffleSplit
from sklearn.preprocessing import LabelEncoder

from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC

from sklearn.model_selection import cross_val_predict, cross_val_score
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, precision_score, recall_score, f1_score
from sklearn.metrics import classification_report, confusion_matrix, precision_recall_fscore_support

from sklearn.pipeline import Pipeline, FeatureUnion

from xgboost import XGBClassifier
import pickle


In [2]:
import sys
sys.path.append(f'../../src/python')

from balance_split_data import create_label, balance_unique_id, \
create_balanced_excluded, create_train_test_excluded, group_by_case, create_feature_label

from grid_search import gridsearch, fit_best_model_train, evaluate, fit_best_model_test, get_feature_importance_cv

from ml_model import train_model_cross_val, train_model_test, \
get_feature_importance, get_feature_importance10

### Load data

In [3]:
df = pd.read_json(r"../../data/processed/pre-processed_2023_03_11.json")

df.drop_duplicates(subset=['article_new', 'case_num', 'file', 'section_fin'], keep='first', inplace=True)
df=df.reset_index(drop=True)

We need to remove some obvious words like 'european commission', 'statement objection', etc. These are most likely frequest terms as well.

In [4]:
# Define a regular expression pattern to match the unwanted words
remove_words = ['statement objection', 'en', 'phase ii', 'rfi', 'request information',
                'european commission']
remove_pattern = re.compile(r'\b(' + '|'.join(remove_words) + r')\b', flags=re.IGNORECASE)
df['text_clean'] = df['text_clean'].replace(remove_pattern, '', regex=True)

### Balance data and split to train and test set

In [5]:
label_name = "phase2"

In [6]:
df1 = create_label(df, label_name)
df_unique = balance_unique_id(df1)
df_balanced_unique, df_balanced, df_excluded = create_balanced_excluded(df_unique, df1)
df_train, df_test, df_test1 = create_train_test_excluded(df_balanced, df_balanced_unique, df_excluded)
df_train_grouped, df_test_grouped, df_test1_grouped, df_excluded_grouped = group_by_case(df_train, df_test, df_test1, df_excluded)

Total decisions: 1583
0    1485
1      98
Name: label, dtype: int64
Balancing...
Total decisions: 196
Labels distribution: 
 0    98
1    98
Name: 0, dtype: int64
Training set shape: (157,) (157,)
Test set shape: (40,) (40,)
Creating df_train 1: 78 0: 80
Creating df_test1 1: 21 0: 20
Creating df_test concatenated with df_excluded with len: 1386
Creating df_test 1: 21 0: 1406
Grouping df_train by case_num 1: 78 0: 80
Grouping df_test by case_num 1: 21 0: 1406
Grouping df_test1 by case_num 1: 21 0: 20
Grouping df_excluded by case_num with len: 1386


In [7]:
X_train, y_train, X_test, y_test, X_test1, y_test1, X_test2, y_test2 = create_feature_label(df_train_grouped, df_test_grouped, df_test1_grouped, df_excluded_grouped)

### Hyperparameter tuning with GridSearch Cross Validation

In [8]:
# Define model
solver='liblinear'
model = LogisticRegression(solver=solver)

# Create a pipeline with TfidfVectorizer and Model
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(analyzer='word', max_features=5000, stop_words='english')),
    ('clf', model)

])

# Define the grid of hyperparameters to search over
parameters = {
    'tfidf__ngram_range': [(2,2),(2,3), (2,4), (3,3), (3,4)], #(1,2),(1,1),(1,3),
    #'tfidf__analyzer': ('word', 'char'),
    #'tfidf__lowercase': (True, False),
    'tfidf__max_df': [[0.01, 0.10, 0.2]], # (0.01, 1.0), # ignore words that occur as more than x% of corpus
    # 'tfidf__min_df': (1, 2, 3), # we need to see a word at least (once, twice, thrice) in a document
    'tfidf__use_idf': (False, True), # use inverse document frequency weighting
    #'tfidf__sublinear_tf': (False, True),
    'tfidf__binary': (False, True), #set term frequency binary (all non-zero terms are set to 1)
    'tfidf__norm': ('l1', 'l2'), #norm used to normalize term vectors
    # 'tfidf__max_features': (None, 2000, 5000),
    #'tfidf__stop_words': (None, 'english'),

    # 'clf__solver': ['liblinear', 'lbfgs'],
    'clf__C':(0.1, 1, 5) # penalty parameter for the model
}

In [10]:
grid_search, best_parameters = gridsearch(pipeline, parameters, X_train, y_train, cv=3, scoring='f1')

Fitting 3 folds for each of 768 candidates, totalling 2304 fits
done in 4093.514s
Best cross-validation score:  0.9325116713352007
Best parameters set:
	clf__C: 5
	tfidf__binary: True
	tfidf__max_df: 0.25
	tfidf__ngram_range: (1, 3)
	tfidf__norm: 'l2'
	tfidf__use_idf: True


##### Model fitting and evaluation

In [74]:
pipeline_cv, y_predict_cv = fit_best_model_train(X_train, y_train, model, best_parameters, cv=3)

fitting the best model
Accuracy: 0.930379746835443

Classification report:
               precision    recall  f1-score   support

           0       0.94      0.93      0.93        80
           1       0.92      0.94      0.93        78

    accuracy                           0.93       158
   macro avg       0.93      0.93      0.93       158
weighted avg       0.93      0.93      0.93       158


CR: (0.9303797468354431, 0.930448717948718, 0.9303769578976887, None)

Confusion matrix:
 [[74  6]
 [ 5 73]] 

_______________________


 Accuracy: 93.04 
 Precision: 0.924 
 Recall: 0.936 
 F1: 0.930


In [76]:
evaluate(y_train, y_predict_cv)

Accuracy: 0.930379746835443

Classification report:
               precision    recall  f1-score   support

           0       0.94      0.93      0.93        80
           1       0.92      0.94      0.93        78

    accuracy                           0.93       158
   macro avg       0.93      0.93      0.93       158
weighted avg       0.93      0.93      0.93       158


CR: (0.9303797468354431, 0.930448717948718, 0.9303769578976887, None)

Confusion matrix:
 [[74  6]
 [ 5 73]] 

_______________________


 Accuracy: 93.04 
 Precision: 0.924 
 Recall: 0.936 
 F1: 0.930


Saving the features and coefficients into a dataframe determined by gridsearchCV best_parameters.

In [102]:
df_features_cv = get_feature_importance_cv(pipeline_cv)
df_features_cv.to_excel('../../../../output/features_logit_cv_full.xlsx', index=True)
df_features_cv.head()

,feature,importance
1691,europese,-0.805246
600,commissie,-0.805246
429,brussel,-0.805246
1692,europese commissie,-0.805246
434,bruxelles,-0.790418


In [75]:
y_predict_cv_test = fit_best_model_test(X_test, y_test, pipeline_cv)

testing on test set
Accuracy: 0.9482517482517483

Classification report:
               precision    recall  f1-score   support

           0       1.00      0.95      0.97      1409
           1       0.20      0.86      0.33        21

    accuracy                           0.95      1430
   macro avg       0.60      0.90      0.65      1430
weighted avg       0.99      0.95      0.96      1430


CR: (0.6000050272729558, 0.9033762546892425, 0.6501818181818182, None)

Confusion matrix:
 [[1338   71]
 [   3   18]] 

_______________________


 Accuracy: 94.83 
 Precision: 0.202 
 Recall: 0.857 
 F1: 0.327


### Model training (with 10-fold CV) using *best/chosen* parameters from GridSearch

Set parameters based on grdisearch or choose parameters to set

In [66]:
for param_name in sorted(parameters.keys()):
        print("\t%s: %r" % (param_name, best_parameters[param_name]))

	clf__C: 5
	tfidf__binary: True
	tfidf__max_df: 0.25
	tfidf__ngram_range: (1, 3)
	tfidf__norm: 'l2'
	tfidf__use_idf: True


In [312]:
vec = ('wordvec', TfidfVectorizer(analyzer = 'word', max_features=5000, stop_words='english', ngram_range = (3,3), binary = True, max_df = 0.01, norm = 'l2', use_idf = True))
c = 5
solver = 'liblinear'
model=LogisticRegression(C=c, solver=solver)

In [313]:
pipeline_train, trained_model, y_predict_train = train_model_cross_val(X_train, y_train, vec=vec, model=model, cv=10)

***10-fold cross-validation***
Accuracy: 0.6329113924050633

Classification report:
               precision    recall  f1-score   support

           0       0.59      0.94      0.72        80
           1       0.83      0.32      0.46        78

    accuracy                           0.63       158
   macro avg       0.71      0.63      0.59       158
weighted avg       0.71      0.63      0.59       158


CR: (0.7096354166666667, 0.6290064102564102, 0.5920584045584045, None)

Confusion matrix:
 [[75  5]
 [53 25]] 

_______________________


 Accuracy: 63.29 
 Precision: 0.833 
 Recall: 0.321 
 F1: 0.463


In [314]:
pipeline_test, y_predict_test = train_model_test(X_train, y_train, X_test, y_test, model=model, vec=vec)

***testing on test set***
Accuracy: 0.9657342657342657

Classification report:
               precision    recall  f1-score   support

           0       0.99      0.98      0.98      1409
           1       0.11      0.19      0.14        21

    accuracy                           0.97      1430
   macro avg       0.55      0.58      0.56      1430
weighted avg       0.97      0.97      0.97      1430


CR: (0.5494579945799458, 0.5838825239109129, 0.561434803562599, None)

Confusion matrix:
 [[1377   32]
 [  17    4]] 

_______________________


 Accuracy: 96.57 
 Precision: 0.111 
 Recall: 0.190 
 F1: 0.140


### Feature importance

In [319]:
df_features_train, feature_importance, feature_names = get_feature_importance(trained_model, pipeline_train)
df_features_train.head(10)

,feature,importance,abs_importance
2401,plastic pa ep,1.320381,1.320381
2353,plastic glass organic,-1.040707,1.040707
2988,platform collude coverage,-1.040707,1.040707
4909,player coverage region,-1.040707,1.040707
3603,platform technical manufacturing,-1.040707,1.040707
3604,platform technology chrysler,-1.040707,1.040707
3294,platform larger range,-1.040707,1.040707
4642,player bat jt,-1.040707,1.040707
2207,plasma gas traditional,-1.040707,1.040707
4978,player depends business,-1.040707,1.040707


In [320]:
# df_features_train = df_features_train.sort_values(by='abs_importance', ascending=False)
# df_features_train_top10 = df_features_train.nlargest(10, 'abs_importance') #importance

# fig, ax = plt.subplots(figsize=(8, 6))
# ax.bar(df_features_train_top10['feature'], df_features_train_top10['importance'], color='darkgreen')
# ax.set_xlabel('Importance')
# ax.set_title('Top 10 Features by Absolute Importance')

# plt.show()


Get features for label = 1 and label = 0

In [321]:
df_features_train1, df_features_train0 = get_feature_importance10(df_features_train)

In [322]:
# Sort the features by absolute importance for label=1
df_features_train1 = df_features_train1.sort_values(by='abs_importance', ascending=False)
df_features_train1_top10 = df_features_train1.nlargest(10, 'abs_importance')

# Sort the features by absolute importance for label=0
df_features_train0 = df_features_train0.sort_values(by='abs_importance', ascending=False)
df_features_train0_top10 = df_features_train0.nlargest(10, 'abs_importance')

# Create a combined bar chart
fig, ax = plt.subplots(figsize=(12, 8))
ax.barh(df_features_train1_top10['feature'], df_features_train1_top10['importance'], color='maroon', label='Phase 2=1')
ax.barh(df_features_train0_top10['feature'], df_features_train0_top10['importance'], color='darkblue', label='Phase 2=0')
ax.set_xlabel('Features')
ax.set_ylabel('Importance')
ax.set_title('Top 10 Features by Importance')
ax.legend()
plt.xticks(rotation=90)
plt.show()


In [306]:
df_features_train.to_excel('../../../../output/features_logit_train_full.xlsx', index=True)
# df_features_train1.to_excel('../../../../output/features_logit_train1_full.xlsx', index=True)
# df_features_train0.to_excel('../../../../output/features_logit_train0_full.xlsx', index=True)